In [1]:
import pandas as pd
import numpy as np
import pathlib
import os

from uszipcode import SearchEngine

In [258]:
state_map = {
    "AK": "Alaska",
    "AL": "Alabama",
    "AR": "Arkansas",
    "AZ": "Arizona",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DC": "District of Columbia",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "IA": "Iowa",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "MA": "Massachusetts",
    "MD": "Maryland",
    "ME": "Maine",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MO": "Missouri",
    "MS": "Mississippi",
    "MT": "Montana",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "NE": "Nebraska",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NV": "Nevada",
    "NY": "New York",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VA": "Virginia",
    "VT": "Vermont",
    "WA": "Washington",
    "WI": "Wisconsin",
    "WV": "West Virginia",
    "WY": "Wyoming",
}

state_list = list(state_map.keys())

In [276]:
os.getcwd()
os.chdir('/Users/tylerlewris/Documents/DataVisualization/dash-medical-provider-charges')

In [264]:
data_dict = {}
for state in state_list:
    #p = os.getcwd().split(os.path.sep)
    csv_path = "df_{}_lat_lon.csv".format(state)
    state_data = pd.read_csv(csv_path)
    data_dict[state] = state_data

In [269]:
data_dict.values()

dict_values([                                        DRG Definition  Provider Id  \
0    003 - ECMO OR TRACH W MV >96 HRS OR PDX EXC FA...        20001   
1    025 - CRANIOTOMY & ENDOVASCULAR INTRACRANIAL P...        20001   
2                   038 - EXTRACRANIAL PROCEDURES W CC        20017   
3             039 - EXTRACRANIAL PROCEDURES W/O CC/MCC        20001   
4             039 - EXTRACRANIAL PROCEDURES W/O CC/MCC        20017   
5    064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...        20001   
6    064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...        20017   
7    064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...        20026   
8    065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...        20001   
9    065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...        20006   
10   065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...        20012   
11   065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...        20017   
12   065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...        20

In [2]:
#Grab data file
path = os.getcwd()
os.chdir(path+'/data')

In [206]:
all_data = pd.read_csv("Hospital_General_Information.csv")

In [97]:
all_data.isna().sum()

Facility ID                                                         0
Facility Name                                                       0
Address                                                             0
City                                                                0
State                                                               0
ZIP Code                                                            0
County Name                                                         0
Phone Number                                                        0
Hospital Type                                                       0
Hospital Ownership                                                  0
Emergency Services                                                  0
Meets criteria for meaningful use of EHRs                        1644
Hospital overall rating                                             0
Hospital overall rating footnote                                 3518
Mortality national c

In [207]:
#For purposes of this project, lets build a data frame that only includes columns with no missing values
#Select non missing value columns
x = all_data.isna().sum()
xx = x[x == 0]

#Pull out dataframe of only columns that have no missing values
column_list = []
column_list.append('Location') #adding back location and removing values after
for i in xx.index:
     column_list.append(i)
        
data = all_data[column_list]        

In [104]:
data[data['Hospital overall rating'] == 'Not Available'].count()

Location                                                1553
Facility ID                                             1761
Facility Name                                           1761
Address                                                 1761
City                                                    1761
State                                                   1761
ZIP Code                                                1761
County Name                                             1761
Phone Number                                            1761
Hospital Type                                           1761
Hospital Ownership                                      1761
Emergency Services                                      1761
Hospital overall rating                                 1761
Mortality national comparison                           1761
Safety of care national comparison                      1761
Readmission national comparison                         1761
Patient experience natio

In [208]:
#Make sure we have no missing Hospital Overall Ratings
clean_data = data.loc[data['Hospital overall rating'] != 'Not Available']

In [148]:
clean_data.loc[data['Mortality national comparison'] == 'Not Available'].count()

Location                                                263
Facility ID                                             279
Facility Name                                           279
Address                                                 279
City                                                    279
State                                                   279
ZIP Code                                                279
County Name                                             279
Phone Number                                            279
Hospital Type                                           279
Hospital Ownership                                      279
Emergency Services                                      279
Hospital overall rating                                 279
Mortality national comparison                           279
Safety of care national comparison                      279
Readmission national comparison                         279
Patient experience national comparison  

In [209]:
#Show how many observations have "Not Available" 
clean_data.isin(['Not Available']).sum()

Location                                                  0
Facility ID                                               0
Facility Name                                             0
Address                                                   0
City                                                      0
State                                                     0
ZIP Code                                                  0
County Name                                               0
Phone Number                                              0
Hospital Type                                             0
Hospital Ownership                                        0
Emergency Services                                        0
Hospital overall rating                                   0
Mortality national comparison                           279
Safety of care national comparison                      974
Readmission national comparison                          53
Patient experience national comparison  

In [220]:
# Generate lat, lon from zip code
search = SearchEngine(
    simple_zipcode=False
)  # set simple_zipcode=False to use rich info database


def generate_lat_lon(df):
    df["lat"] = df["lon"] = np.nan
    zip_code = clean_data["ZIP Code"]
    for ind, item in enumerate(zip_code):
        zipcode = search.by_zipcode(str(item))
        zip = zipcode.to_dict()
        df.loc[ind, "lat"] = zip["lat"]
        df.loc[ind, "lon"] = zip["lng"]
    return df

In [211]:
test1 = clean_data.copy().reset_index()

In [217]:
test1['ZIP Code'] = test1['ZIP Code'].astype(int)

In [221]:
generate_lat_lon(test1)

,index,Location,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,...,Hospital overall rating,Mortality national comparison,Safety of care national comparison,Readmission national comparison,Patient experience national comparison,Effectiveness of care national comparison,Timeliness of care national comparison,Efficient use of medical imaging national comparison,lat,lon
0,7,POINT (-101.713256 39.35759),171370,GOODLAND REGIONAL MEDICAL CENTER,220 WEST SECOND STREET,GOODLAND,KS,67735,SHERMAN,(785) 890-3625,...,3,Same as the national average,Not Available,Same as the national average,Not Available,Not Available,Above the national average,Not Available,39.40,-101.80
1,14,NaN,340132,MARIA PARHAM MEDICAL CENTER,PO BOX 59,HENDERSON,NC,27536,VANCE,(252) 431-3708,...,2,Same as the national average,Same as the national average,Below the national average,Below the national average,Same as the national average,Below the national average,Same as the national average,36.33,-78.41
2,16,NaN,341320,ALLEGHANY COUNTY MEMORIAL HOSPITAL,617 DOCTORS STREET,SPARTA,NC,28675,ALLEGHANY,(336) 372-5511,...,2,Not Available,Not Available,Below the national average,Not Available,Not Available,Above the national average,Same as the national average,36.50,-81.20
3,22,POINT (-76.958232 39.052263),210016,ADVENTIST HEALTHCARE WHITE OAK MEDICAL CENTER,12100 PLUM ORCHARD DRIVE,SILVER SPRING,MD,20904,MONTGOMERY,(301) 891-5651,...,3,Same as the national average,Below the national average,Above the national average,Below the national average,Same as the national average,Below the national average,Same as the national average,39.07,-76.98
4,24,POINT (-81.564666 27.896533),100099,ADVENTHEALTH LAKE WALES,410 S 11TH ST,LAKE WALES,FL,33853,POLK,(863) 676-1433,...,1,Same as the national average,Same as the national average,Below the national average,Below the national average,Same as the national average,Same as the national average,Not Available,27.90,-81.59
5,27,POINT (-74.213787 40.710054),310002,NEWARK BETH ISRAEL MEDICAL CENTER,201 LYONS AVE,NEWARK,NJ,7112,ESSEX,(973) 926-7850,...,3,Same as the national average,Above the national average,Below the national average,Below the national average,Same as the national average,Below the national average,Same as the national average,40.71,-74.21
6,28,POINT (-87.691909 41.770008),140133,HOLY CROSS HOSPITAL,2701 W 68TH STREET,CHICAGO,IL,60629,COOK,(773) 884-9000,...,2,Same as the national average,Above the national average,Below the national average,Below the national average,Same as the national average,Below the national average,Same as the national average,41.78,-87.70
7,31,POINT (-106.636379 48.189571),271316,FRANCES MAHON DEACONESS HOSPITAL,621 3RD ST S,GLASGOW,MT,59230,VALLEY,(406) 228-3500,...,4,Same as the national average,Not Available,Same as the national average,Above the national average,Same as the national average,Above the national average,Same as the national average,48.30,-106.60
8,34,POINT (-81.765071 33.56989),420082,AIKEN REGIONAL MEDICAL CENTER,302 UNIVERSITY PARKWAY,AIKEN,SC,29801,AIKEN,(803) 641-5900,...,2,Below the national average,Same as the national average,Above the national average,Below the national average,Same as the national average,Same as the national average,Same as the national average,33.60,-81.70
9,36,POINT (-84.883951 39.112922),150086,HIGHPOINT HEALTH,600 WILSON CREEK RD,LAWRENCEBURG,IN,47025,DEARBORN,(812) 537-1010,...,4,Same as the national average,Same as the national average,Same as the national average,Above the national average,Same as the national average,Above the national average,Same as the national average,39.20,-84.90


In [251]:
test2 = test1[test1['lat'].isna() == False]

In [271]:
final_df = test2.drop(['Location','index'], axis=1)

In [284]:
final_df['Hospital overall rating'].unique()

array(['3', '2', '1', '4', '5'], dtype=object)

In [281]:
final_df.to_csv('hospital_rating.csv', index=False)

In [ ]:
Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code',
       'County Name', 'Phone Number', 'Hospital Type', 'Hospital Ownership',
       'Emergency Services', 'Meets criteria for meaningful use of EHRs',
       'Hospital overall rating', 'Hospital overall rating footnote',
       'Mortality national comparison',
       'Mortality national comparison footnote',
       'Safety of care national comparison',
       'Safety of care national comparison footnote',
       'Readmission national comparison',
       'Readmission national comparison footnote',
       'Patient experience national comparison',
       'Patient experience national comparison footnote',
       'Effectiveness of care national comparison',
       'Effectiveness of care national comparison footnote',
       'Timeliness of care national comparison',
       'Timeliness of care national comparison footnote',
       'Efficient use of medical imaging national comparison',
       'Efficient use of medical imaging national comparison footnote',
       'Location'],
      dtype='object')